#**Predicting Sleep Stages Using EEG Data**

In [48]:
'''sleep Stages
1.Wakefulness (W):

Characteristics: High-frequency, low-amplitude EEG activity. The individual is awake and alert.
EEG Patterns: Alpha waves (8-13 Hz) are dominant when eyes are closed; beta waves (14-30 Hz) when eyes are open.
Non-Rapid Eye Movement (NREM) Sleep:

2.NREM sleep is further divided into three stages (N1, N2, N3) in modern classifications, but historically, it was divided into four stages. Here we will discuss the modern three-stage division.

      1.Stage N1 (Light Sleep):

Characteristics: Transition between wakefulness and sleep.
EEG Patterns: Low voltage, mixed frequency, with theta waves (4-7 Hz) becoming more prominent.
      2.Stage N2 (Moderate Sleep):

Characteristics: Deeper sleep than N1, but the person can still be easily awakened.
EEG Patterns: Sleep spindles (bursts of 12-14 Hz waves) and K-complexes (sharp high-amplitude waves).
      3.Stage N3 (Deep Sleep, Slow-Wave Sleep):

Characteristics: Deepest stage of NREM sleep, difficult to awaken the person.
EEG Patterns: Delta waves (0.5-4 Hz) are dominant, high amplitude, low frequency.
3.Rapid Eye Movement (REM) Sleep:

Characteristics: Brain activity resembles wakefulness, but the person is asleep. Most dreaming occurs in this stage. The body is paralyzed to prevent acting out dreams.
EEG Patterns: Low voltage, mixed frequency activity, similar to wakefulness.
Sleep Stages in Your Model
Your model is designed to classify these sleep stages based on EEG data. Here's how each stage maps to the classification problem in your model:

Wakefulness (W)
NREM Stage 1 (N1)
NREM Stage 2 (N2)
NREM Stage 3 (N3)
REM Sleep (R).'''

"sleep Stages\n1.Wakefulness (W):\n\nCharacteristics: High-frequency, low-amplitude EEG activity. The individual is awake and alert.\nEEG Patterns: Alpha waves (8-13 Hz) are dominant when eyes are closed; beta waves (14-30 Hz) when eyes are open.\nNon-Rapid Eye Movement (NREM) Sleep:\n\n2.NREM sleep is further divided into three stages (N1, N2, N3) in modern classifications, but historically, it was divided into four stages. Here we will discuss the modern three-stage division.\n\n      1.Stage N1 (Light Sleep):\n\nCharacteristics: Transition between wakefulness and sleep.\nEEG Patterns: Low voltage, mixed frequency, with theta waves (4-7 Hz) becoming more prominent.\n      2.Stage N2 (Moderate Sleep):\n\nCharacteristics: Deeper sleep than N1, but the person can still be easily awakened.\nEEG Patterns: Sleep spindles (bursts of 12-14 Hz waves) and K-complexes (sharp high-amplitude waves).\n      3.Stage N3 (Deep Sleep, Slow-Wave Sleep):\n\nCharacteristics: Deepest stage of NREM sleep, 

#Imports and Libraries:
**pyEDFlib:** This is used for reading the .edf files which store both PSG signals and annotations (hypnograms).

**MNE:** A library used for loading, manipulating, and analyzing EEG data.

In [1]:
!pip install pyEDFlib
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 31.6 MB/s eta 0:00:00


In [2]:
import pyedflib
import numpy as np
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
import os
import glob
import mne
from pyedflib import highlevel
import pandas as pd
from sklearn.preprocessing import StandardScaler
import math
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow as tf
from sklearn.metrics import precision_score, f1_score
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense, Input, LSTM, BatchNormalization, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop


#Stage Dictionary and label mapping:

In [3]:
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4
UNKNOWN = 5

stage_dict = {

    "W": W,
    "N1": N1,
    "N2": N2,
    "N3": N3,
    "REM": REM,
    "UNKNOWN": UNKNOWN
}

class_dict = {

    1: "N1",
    2: "N2",
    3: "N3",
    4: "REM",
    5: "UNKNOWN"
}

ann2label = {


    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4,
    "Sleep stage ?": 5,
    "Movement time": 5

}

EPOCH_SEC_SIZE = 30

#Data Input:
data_dir: Directory path containing the PSG and Hypnogram .edf files.

psg_files: Contains all PSG (EEG) files.

ann_files: Contains all annotation files, corresponding to the hypnograms.

save_folder: Directory path to store the processed EEG data for later use in model training.

test_file_path:give pkl file path after preprocessing edf file to get output.

In [4]:
data_dir = "/content/drive/MyDrive/EEG data/sleep-cassette"
save_folder = '/content/drive/MyDrive/EEG data/save_folder'
test_file_path = '/content/drive/MyDrive/EEG data/test/processed_data_0.pkl'

In [5]:
scaler = StandardScaler()

In [6]:
psg_files = glob.glob(data_dir + "/*PSG.edf")
psg_files.sort()
ann_files = glob.glob(data_dir + "/*Hypnogram.edf")
ann_files.sort()

In [ ]:
psg_files = np.asarray(psg_files)
ann_files = np.asarray(ann_files)

In [ ]:
assert len(psg_files) > 0, "No PSG files found in the specified directory."
assert len(ann_files) > 0, "No Hypnogram files found in the specified directory."

#Preprocessing Pipeline:

**Loading the PSG Data:**

MNE's read_raw_edf is used to load the raw EEG data from the PSG files.
The EEG channel selected for this analysis is EEG Fpz-Cz, which is commonly used in sleep studies for detecting brain waves during different sleep stages.

**Normalization:**

EEG data is normalized using StandardScaler.

**Annotations:**

Hypnogram data is read using pyEDFlib and annotations are extracted to associate the correct sleep stage with corresponding time windows in the EEG signal.
Each annotation contains a start time (onset), duration (duration), and the stage label (ann). These are converted into machine-readable labels using the ann2label mapping dictionary.

**Epoching:**

The raw EEG data is split into fixed-sized epochs, with each epoch corresponding to a 30-second window of EEG data (as defined by EPOCH_SEC_SIZE).
The raw signal data is chunked into epochs of shape (epoch_samples, channels).

**Label Extraction:**

Each epoch is labeled based on the associated sleep stage from the hypnogram.
The data is filtered to remove "Wake" periods (stage_dict["W"]), as only sleep periods are considered for model training.

**Saving Processed Data:**

For each subject (i.e., each PSG file), the preprocessed data (x: EEG epochs, y: sleep stage labels) is saved as a .pkl file using the pickle library for later use in model training.

In [ ]:
for i in range(153):
    raw = mne.io.read_raw_edf(psg_files[i], preload=False, verbose='WARNING')
    sampling_rate = raw.info['sfreq']

    raw.pick(['EEG Fpz-Cz'])
    raw_data = raw.get_data()
    raw_data = scaler.fit_transform(raw_data.T).T

    # Load hypnogram and extract annotations
    with pyedflib.EdfReader(ann_files[i]) as ann_file:
        ann_data = ann_file.readAnnotations()

    labels = []
    label_idx = []

    for onset, duration, ann in zip(ann_data[0], ann_data[1], ann_data[2]):
        label = ann2label.get(ann, UNKNOWN)
        if label != UNKNOWN:
            duration_epoch = int(duration / EPOCH_SEC_SIZE)
            labels.extend([label] * duration_epoch)
            start_idx = int(onset * sampling_rate)
            end_idx = start_idx + int(duration * sampling_rate)
            label_idx.append(np.arange(start_idx, end_idx))

    # Concatenate labels
    labels = np.hstack(labels)
    label_idx = np.hstack(label_idx)
    select_idx = label_idx

    raw_ch = raw_data[:, select_idx].T

    n_epochs = len(raw_ch) // int(EPOCH_SEC_SIZE * sampling_rate)
    x = np.array(np.split(raw_ch, n_epochs)).astype(np.float32)
    y = labels.astype(np.int32)

    assert len(x) == len(y)

    nw_idx = np.where(y != stage_dict["W"])[0]

     # Subset the data to include only non-wake periods
    x = x[nw_idx]
    y = y[nw_idx]

    data = {'x': x, 'y': y}
    save_path = os.path.join(save_folder, f'processed_data_{i}.pkl')
    with open(save_path, 'wb') as f:
        pickle.dump(data, f)

<ipython-input-54-32f3e1c6fb28>:2: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_files[i], preload=False, verbose='WARNING')
<ipython-input-54-32f3e1c6fb28>:2: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_files[i], preload=False, verbose='WARNING')
<ipython-input-54-32f3e1c6fb28>:2: RuntimeWarning: Highpass cutoff frequency 16.0 is greater than lowpass cutoff frequency 0.7, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(psg_files[i], preload=False, verbose='WARNING')
<ipython-input-54-32f3e1c6fb28>:2: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_files[i], preload=False, verbose='WARNING')
<ipython-input-54-32f3e1c6fb28>:2: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.

#Data Generator Functions

In [27]:
file_list = [filename for filename in os.listdir(save_folder) if filename.endswith(".pkl")]

In [28]:
def load_data_from_file(filename):
    with open(os.path.join(save_folder, filename), 'rb') as f:
        data = pickle.load(f)
        X = data['x']
        y = data['y']
    return X, y

In [29]:
def data_generator(file_list, batch_size=64):
    while True:
        for filename in file_list:
            X, y = load_data_from_file(filename)
            X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
            num_samples = X.shape[0]
            num_batches = (num_samples //  batch_size)
            for i in range(num_batches):
                start_idx = i * batch_size
                end_idx = (i + 1) * batch_size
                yield X[start_idx:end_idx], y[start_idx:end_idx]

In [30]:
total_samples = sum(load_data_from_file(filename)[0].shape[0] for filename in file_list)

In [31]:
batch_size = 64

In [32]:
if total_samples < batch_size:
    steps_per_epoch = 1  # Set steps_per_epoch to 1 to process all samples
else:
    steps_per_epoch = total_samples // batch_size

 # Model Architecture

In [33]:
all_labels = np.concatenate([load_data_from_file(filename)[1] for filename in file_list])
num_classes = len(np.unique(all_labels))

In [34]:
sample_X, _ = load_data_from_file(file_list[0])
input_shape = (sample_X.shape[1], sample_X.shape[2])  # Shape should be (time_steps, channels)
num_classes = len(np.unique(all_labels))

In [35]:
input_shape

(3000, 1)

#Training and Evaluation

In [47]:
input_shape = (3000, 1)

# Input layer
inputs = Input(shape=input_shape)

# CNN layers
conv1 = Conv1D(filters=128, kernel_size=50, strides=1, activation='relu', padding='same')(inputs)
conv1 = BatchNormalization()(conv1)
conv2 = Conv1D(filters=64, kernel_size=20, strides=1, activation='relu', padding='same')(conv1)
conv2 = BatchNormalization()(conv2)
pool1 = MaxPooling1D(pool_size=10, strides=5)(conv2)
dropout1 = Dropout(0.4)(pool1)


conv3 = Conv1D(filters=32, kernel_size=10, strides=1, activation='relu', padding='same')(dropout1)
conv3 = BatchNormalization()(conv3)
conv4 = Conv1D(filters=16, kernel_size=5, strides=1, activation='relu', padding='same')(conv3)
conv4 = BatchNormalization()(conv4)
pool2 = MaxPooling1D(pool_size=5, strides=2)(conv4)
dropout2 = Dropout(0.4)(pool2)

#GRU layers
gru1 = GRU(50, return_sequences=True)(dropout2)
gru2 = GRU(50)(gru1)

# Fully connected layers
flatten = Flatten()(gru2)
fc1 = Dense(64, activation='relu')(flatten)
fc2 = Dense(32, activation='relu')(fc1)
outputs = Dense(5, activation='softmax')(fc2)

# Compile the model
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 3000, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_11 (Conv1D)                   │ (None, 3000, 128)           │           6,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 3000, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_12 (Conv1D)                   │ (None, 3000, 64)            │         163,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 3000, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_7 (MaxPooling1D)       │ (None, 599, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 599, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_13 (Conv1D)                   │ (None, 599, 32)             │          20,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 599, 32)             │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_14 (Conv1D)                   │ (None, 599, 16)             │           2,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 599, 16)             │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_8 (MaxPooling1D)       │ (None, 298, 16)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 298, 16)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_4 (GRU)                          │ (None, 298, 50)             │          10,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_5 (GRU)                          │ (None, 50)                  │          15,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 64)                  │           3,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 225,489 (880.82 KB)

 Trainable params: 225,009 (878.94 KB)

 Non-trainable params: 480 (1.88 KB)

In [38]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

In [39]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    'best_model.keras',
    save_best_only=True,
    verbose=1
)

In [40]:
history = model.fit(
    data_generator(file_list, batch_size=batch_size),
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    callbacks=[lr_scheduler, checkpoint]
)


Epoch 1/10
2013/2013 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.5925 - loss: 0.9776
Epoch 1: accuracy improved from -inf to 0.64121, saving model to best_model.keras
2013/2013 ━━━━━━━━━━━━━━━━━━━━ 201s 98ms/step - accuracy: 0.5925 - loss: 0.9775 - learning_rate: 0.0010
Epoch 2/10
2013/2013 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.7601 - loss: 0.5977
Epoch 2: accuracy improved from 0.64121 to 0.74214, saving model to best_model.keras
2013/2013 ━━━━━━━━━━━━━━━━━━━━ 198s 98ms/step - accuracy: 0.7601 - loss: 0.5977 - learning_rate: 0.0010
Epoch 3/10
2013/2013 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.7822 - loss: 0.5497
Epoch 3: accuracy improved from 0.74214 to 0.76309, saving model to best_model.keras
2013/2013 ━━━━━━━━━━━━━━━━━━━━ 199s 99ms/step - accuracy: 0.7822 - loss: 0.5497 - learning_rate: 0.0010
Epoch 4/10
2013/2013 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.7879 - loss: 0.5344
Epoch 4: accuracy improved from 0.76309 to 0.77384, saving model to best_model.k

# evaluation


In [41]:
def test_data_generator(file_list, batch_size=64):
    while True:
        for filename in file_list:
            X, y = load_data_from_file(filename)
            X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
            num_samples = X.shape[0]
            num_batches = (num_samples // batch_size)
            for i in range(num_batches):
                start_idx = i * batch_size
                end_idx = (i + 1) * batch_size
                yield X[start_idx:end_idx], y[start_idx:end_idx]




In [43]:
def compute_metrics(test_gen, steps_per_epoch):
    y_true = []
    y_pred = []

    for _ in range(steps_per_epoch):
        X_batch, y_batch = next(test_gen)
        y_true.extend(y_batch)
        y_pred_probs = model.predict(X_batch)
        y_pred.extend(np.argmax(y_pred_probs, axis=1))

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    precision = precision_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    return precision, f1

# Prepare test data generator
test_file_list = [filename for filename in os.listdir(save_folder) if filename.endswith(".pkl")]
test_gen = test_data_generator(test_file_list, batch_size=64)

# Calculate steps per epoch
total_test_samples = sum(load_data_from_file(filename)[0].shape[0] for filename in test_file_list)
steps_per_epoch = total_test_samples // 64

# Compute metrics
precision, f1 = compute_metrics(test_gen, steps_per_epoch)
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━

Key Metrics
The final metrics of the trained model are as follows:

Precision: 0.7055

F1 Score: 0.6383

Accuracy: 0.7875

In [46]:
import pickle
import os
import numpy as np

def load_preprocessed_data(file_path):

    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        X = data['x']
        y = data.get('y', None)
    return X, y

def predict_on_preprocessed_data(file_path, model):


    X, y_true = load_preprocessed_data(file_path)
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

    # Predict the sleep stages
    predictions_probs = model.predict(X)
    predictions = np.argmax(predictions_probs, axis=1)

    return predictions, y_true

predicted_labels, true_labels = predict_on_preprocessed_data(test_file_path, model)

if true_labels is not None:
    accuracy = np.mean(predicted_labels == true_labels)
    print(f"Accuracy on test data: {accuracy:.4f}")

# Print predicted sleep stages
predicted_sleep_stages = [class_dict[p] for p in predicted_labels]
print("Predicted Sleep Stages:", predicted_sleep_stages)

# Print true sleep stages
if true_labels is not None:
    true_sleep_stages = [class_dict[t] for t in true_labels]
    print("True Sleep Stages:    ", true_sleep_stages)


21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Accuracy on test data: 0.7427
Predicted Sleep Stages: ['REM', 'REM', 'REM', 'REM', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N2', 'N3', 'N3', 'N3', 'N2', 'N3', 'N2', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N1', 'REM', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 